In [2]:
import pandas as pd
import math

In [5]:
df = pd.read_json("./experiment_queries_execution_boost_rerank2.json", orient="records")
df
    

,method_id,user_id,class_id,query_id,class_category,rr,time
0,single_elastic_bm25_boost_0.25,1,20825628,user_query_0,instances_100_1000,0.000000,0.110425
1,single_elastic_bm25_boost_0.25,1,20825628,user_query_1,instances_100_1000,0.000000,0.071166
2,single_elastic_bm25_boost_0.25,1,20825628,user_query_2,instances_100_1000,0.000000,0.051668
3,single_elastic_bm25_boost_0.25,1,20825628,user_query_3,instances_100_1000,0.000000,0.048177
4,single_elastic_bm25_boost_0.25,1,20825628,user_query_4,instances_100_1000,0.000000,0.049556
...,...,...,...,...,...,...,...
298579,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.000000,0.091356
298580,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.000000,0.087534
298581,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_3,instances_100000_1000000000,1.000000,0.090200
298582,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.333333,0.096017


In [6]:
# Number of methods
df["method_id"].nunique()

198

# Overall

In [7]:
df_thin = df[["method_id", "rr", "time"]]
df_thin.head()

,method_id,rr,time
0,single_elastic_bm25_boost_0.25,0.0,0.110425
1,single_elastic_bm25_boost_0.25,0.0,0.071166
2,single_elastic_bm25_boost_0.25,0.0,0.051668
3,single_elastic_bm25_boost_0.25,0.0,0.048177
4,single_elastic_bm25_boost_0.25,0.0,0.049556


In [8]:
df_overall =  df_thin.groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False)
print(df_overall.to_string())

                                                                            method_id        rr      time
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  0.680024  0.174688
157                       tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.5  0.678376  0.181747
145                       tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.5  0.678245  0.177712
106   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.5  0.677917  0.179724
97    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5  0.677313  0.180777
112   triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1_boost_0.5  0.675335  0.178431
163                       tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.5  0.674687  0.178412
139                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5  0.674600  0.179510
85    triple_fusion_qdrant_dense_0.4_x_qdrant_

### Overall only tuples

In [9]:
df_overall_tuple_only = df_overall[df_overall["method_id"].str.contains("tuple")]
df_overall_tuple_only

,method_id,rr,time
151,tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_...,0.680024,0.174688
157,tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_...,0.678376,0.181747
145,tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_...,0.678245,0.177712
163,tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_...,0.674687,0.178412
139,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,0.674600,0.179510
...,...,...,...
118,tuple_fusion_qdrant_dense_0.1_x_elastic_bm25_0...,0.410472,0.179015
174,tuple_fusion_qdrant_sparse_0.2_x_elastic_bm25_...,0.374273,0.106833
123,tuple_fusion_qdrant_dense_0.2_x_elastic_bm25_0...,0.373648,0.179060
117,tuple_fusion_qdrant_dense_0.1_x_elastic_bm25_0...,0.341925,0.179892


### Overall only triples

In [10]:
df_overall_triple_only = df_overall[df_overall["method_id"].str.contains("triple")]
print(df_overall_triple_only.to_string())

                                                                            method_id        rr      time
106   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.5  0.677917  0.179724
97    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5  0.677313  0.180777
112   triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1_boost_0.5  0.675335  0.178431
85    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.5  0.673242  0.179853
103   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.2_x_elastic_bm25_0.2_boost_0.5  0.671694  0.179477
94    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.3_x_elastic_bm25_0.2_boost_0.5  0.670959  0.180092
115   triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm25_0.1_boost_0.5  0.670185  0.180243
82    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2_boost_0.5  0.668385  0.180227
70    triple_fusion_qdrant_dense_0.3_x_qdrant_

# Overall per category



In [11]:
df_thin_per_category = df[["method_id", "class_category", "rr", "time"]]
df_thin_per_category

,method_id,class_category,rr,time
0,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.110425
1,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.071166
2,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.051668
3,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.048177
4,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.049556
...,...,...,...,...
298579,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.091356
298580,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.087534
298581,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.090200
298582,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,0.333333,0.096017


In [12]:
df_overall_per_category = df_thin_per_category.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category","rr"], ascending=False)
print(df_overall_per_category.to_string())

                                                                             method_id               class_category        rr      time
1167                       tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.5           instances_100_1000  0.654990  0.179703
1119                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5           instances_100_1000  0.654200  0.181818
1215                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5           instances_100_1000  0.652068  0.176955
783    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5           instances_100_1000  0.651780  0.184530
63                                                      single_qdrant_sparse_boost_0.5           instances_100_1000  0.651250  0.091563
975                        tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9_boost_0.5           instances_100_1000  0.651244  0.183400
687    triple_fusion_qdrant_dense_0.4_x_qdrant_s

### Only instance categories

In [13]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("instances")].to_string())

                                                                             method_id               class_category        rr      time
1167                       tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.5           instances_100_1000  0.654990  0.179703
1119                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5           instances_100_1000  0.654200  0.181818
1215                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5           instances_100_1000  0.652068  0.176955
783    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5           instances_100_1000  0.651780  0.184530
63                                                      single_qdrant_sparse_boost_0.5           instances_100_1000  0.651250  0.091563
975                        tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9_boost_0.5           instances_100_1000  0.651244  0.183400
687    triple_fusion_qdrant_dense_0.4_x_qdrant_s

### Only ancestors cetegories

In [14]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("ancestors")].to_string())

                                                                             method_id            class_category        rr      time
25                                                      single_qdrant_dense_boost_0.25  ancestors_180_1000000000  0.812500  0.187582
33                                                       single_qdrant_dense_boost_0.5  ancestors_180_1000000000  0.812500  0.182235
1345                      tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1_boost_0.25  ancestors_180_1000000000  0.812500  0.186769
1353                       tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1_boost_0.5  ancestors_180_1000000000  0.812500  0.188564
1297                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.25  ancestors_180_1000000000  0.809295  0.187146
1305                       tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.5  ancestors_180_1000000000  0.809295  0.185529
913   triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm

### Only children

In [15]:
print(df_overall_per_category[df_overall_per_category["class_category"].str.startswith("children")].to_string())

                                                                             method_id              class_category        rr      time
1211                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5       children_10000_100000  0.653882  0.178834
683    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.5       children_10000_100000  0.653256  0.181306
563    triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1_boost_0.5       children_10000_100000  0.652858  0.180869
923    triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm25_0.1_boost_0.5       children_10000_100000  0.651299  0.184114
659    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2_boost_0.5       children_10000_100000  0.649357  0.183094
1115                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5       children_10000_100000  0.649317  0.183791
1019                       tuple_fusion_qdrant_dense_0.

# Overall - only mechanic queries

In [16]:
df_only_mech_queries = df[df["query_id"].str.startswith("mech")]
df_only_mech_queries

,method_id,user_id,class_id,query_id,class_category,rr,time
672,single_elastic_bm25_boost_0.25,0,20825628,mech_label,instances_100_1000,1.000000,0.050921
673,single_elastic_bm25_boost_0.25,0,20825628,mech_desc,instances_100_1000,0.000000,0.118487
674,single_elastic_bm25_boost_0.25,0,20825628,mech_label_desc,instances_100_1000,0.500000,0.149827
675,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_0,instances_100_1000,0.055556,0.120693
676,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_1,instances_100_1000,0.000000,0.097450
...,...,...,...,...,...,...,...
298579,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.000000,0.091356
298580,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.000000,0.087534
298581,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_3,instances_100000_1000000000,1.000000,0.090200
298582,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.333333,0.096017


In [17]:
print(df_only_mech_queries[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())


                                                                            method_id        rr      time
139                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5  0.926017  0.188597
106   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.5  0.925982  0.188223
145                       tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.5  0.925172  0.186426
97    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5  0.924912  0.190064
85    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.5  0.924826  0.189123
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  0.924436  0.181888
157                       tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.5  0.922380  0.190916
82    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2_boost_0.5  0.921427  0.189997
70    triple_fusion_qdrant_dense_0.3_x_qdrant_

### Only labels

In [18]:
df_only_mech_queries_labels_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_label"])]
df_only_mech_queries_labels_only

,method_id,user_id,class_id,query_id,class_category,rr,time
672,single_elastic_bm25_boost_0.25,0,20825628,mech_label,instances_100_1000,1.0,0.050921
681,single_elastic_bm25_boost_0.25,0,2066754,mech_label,instances_1000_10000,1.0,0.068638
690,single_elastic_bm25_boost_0.25,0,34038,mech_label,instances_10000_100000,1.0,0.048768
699,single_elastic_bm25_boost_0.25,0,7725634,mech_label,instances_100000_1000000000,1.0,0.051894
708,single_elastic_bm25_boost_0.25,0,602884,mech_label,children_10000_100000,1.0,0.075789
...,...,...,...,...,...,...,...
298491,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,8016240,mech_label,instances_100_1000,1.0,0.058667
298500,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,23442,mech_label,instances_100000_1000000000,1.0,0.054054
298509,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,30022,mech_label,instances_1000_10000,1.0,0.070175
298518,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,130019,mech_label,instances_10000_100000,1.0,0.064400


In [19]:
print(df_only_mech_queries_labels_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr      time
157                       tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.5  0.991071  0.139908
106   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.5  0.991071  0.134191
28    triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.7_x_elastic_bm25_0.2_boost_0.5  0.991071  0.138054
30   triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1_boost_0.25  0.991071  0.137312
127                       tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8_boost_0.5  0.991071  0.138777
126                      tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8_boost_0.25  0.991071  0.136078
163                       tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.5  0.991071  0.133427
70    triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1_boost_0.5  0.991071  0.134484
120                      tuple_fusion_qdrant_d

### Only descriptions

In [20]:
df_only_mech_queries_desc_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_desc"])]
df_only_mech_queries_desc_only

,method_id,user_id,class_id,query_id,class_category,rr,time
673,single_elastic_bm25_boost_0.25,0,20825628,mech_desc,instances_100_1000,0.000000,0.118487
682,single_elastic_bm25_boost_0.25,0,2066754,mech_desc,instances_1000_10000,0.333333,0.101544
691,single_elastic_bm25_boost_0.25,0,34038,mech_desc,instances_10000_100000,0.000000,0.088817
700,single_elastic_bm25_boost_0.25,0,7725634,mech_desc,instances_100000_1000000000,0.250000,0.054690
709,single_elastic_bm25_boost_0.25,0,602884,mech_desc,children_10000_100000,0.000000,0.114338
...,...,...,...,...,...,...,...
298492,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,8016240,mech_desc,instances_100_1000,1.000000,0.153863
298501,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,23442,mech_desc,instances_100000_1000000000,1.000000,0.113632
298510,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,30022,mech_desc,instances_1000_10000,1.000000,0.080833
298519,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,130019,mech_desc,instances_10000_100000,1.000000,0.141264


In [21]:
print(df_only_mech_queries_desc_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr      time
193                       tuple_fusion_qdrant_sparse_0.8_x_elastic_bm25_0.2_boost_0.5  0.936728  0.115032
7                                                      single_qdrant_sparse_boost_0.5  0.936728  0.092976
121                       tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9_boost_0.5  0.936728  0.193606
31    triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1_boost_0.5  0.936728  0.193186
52    triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.7_x_elastic_bm25_0.1_boost_0.5  0.936728  0.188685
127                       tuple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.8_boost_0.5  0.936728  0.193480
196                       tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1_boost_0.5  0.936728  0.108888
28    triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.7_x_elastic_bm25_0.2_boost_0.5  0.933642  0.191793
133                       tuple_fusion_qdrant_

### Only labels with descriptions

In [22]:
df_only_mech_queries_label_desc_only = df_only_mech_queries[df_only_mech_queries["query_id"].isin(["mech_label_desc"])]
df_only_mech_queries_label_desc_only

,method_id,user_id,class_id,query_id,class_category,rr,time
674,single_elastic_bm25_boost_0.25,0,20825628,mech_label_desc,instances_100_1000,0.500000,0.149827
683,single_elastic_bm25_boost_0.25,0,2066754,mech_label_desc,instances_1000_10000,0.500000,0.124570
692,single_elastic_bm25_boost_0.25,0,34038,mech_label_desc,instances_10000_100000,0.142857,0.114039
701,single_elastic_bm25_boost_0.25,0,7725634,mech_label_desc,instances_100000_1000000000,1.000000,0.094730
710,single_elastic_bm25_boost_0.25,0,602884,mech_label_desc,children_10000_100000,0.166667,0.164923
...,...,...,...,...,...,...,...
298493,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,8016240,mech_label_desc,instances_100_1000,1.000000,0.162009
298502,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,23442,mech_label_desc,instances_100000_1000000000,1.000000,0.140050
298511,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,30022,mech_label_desc,instances_1000_10000,1.000000,0.095538
298520,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,0,130019,mech_label_desc,instances_10000_100000,1.000000,0.148974


In [23]:
print(df_only_mech_queries_label_desc_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr      time
99   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.1_x_elastic_bm25_0.3_boost_0.25  1.000000  0.195920
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  1.000000  0.191013
88    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.1_x_elastic_bm25_0.4_boost_0.5  1.000000  0.199189
85    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.5  1.000000  0.197528
84   triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.25  1.000000  0.201944
82    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2_boost_0.5  1.000000  0.201561
81   triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.4_x_elastic_bm25_0.2_boost_0.25  1.000000  0.201614
148                        tuple_fusion_qdrant_dense_0.6_x_elastic_bm25_0.4_boost_0.5  1.000000  0.202201
79    triple_fusion_qdrant_dense_0.4_x_qdrant_

### Only labels with user queries

In [24]:
df_only_mech_queries_label_user_only = df_only_mech_queries[~df_only_mech_queries["query_id"].isin(["mech_desc", "mech_label", "mech_label_desc"])]
df_only_mech_queries_label_user_only

,method_id,user_id,class_id,query_id,class_category,rr,time
675,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_0,instances_100_1000,0.055556,0.120693
676,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_1,instances_100_1000,0.000000,0.097450
677,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_2,instances_100_1000,0.055556,0.081438
678,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_3,instances_100_1000,0.000000,0.081413
679,single_elastic_bm25_boost_0.25,101,20825628,mech_label_user_query_4,instances_100_1000,0.200000,0.076663
...,...,...,...,...,...,...,...
298579,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_1,instances_100000_1000000000,1.000000,0.091356
298580,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_2,instances_100000_1000000000,1.000000,0.087534
298581,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_3,instances_100000_1000000000,1.000000,0.090200
298582,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,114,23442,mech_label_user_query_4,instances_100000_1000000000,0.333333,0.096017


In [25]:
print(df_only_mech_queries_label_user_only[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr      time
106   triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.5  0.915185  0.191735
139                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5  0.914534  0.191919
145                       tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.5  0.913979  0.189245
97    triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.5  0.913655  0.193553
85    triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1_boost_0.5  0.913549  0.192847
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  0.913063  0.185124
112   triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1_boost_0.5  0.911580  0.191820
157                       tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.5  0.911522  0.194047
67    triple_fusion_qdrant_dense_0.3_x_qdrant_

### Overall per category

In [26]:
df_only_mech_queries_per_category_thin = df_only_mech_queries[["method_id", "class_category", "rr", "time"]]
df_only_mech_queries_per_category_thin

,method_id,class_category,rr,time
672,single_elastic_bm25_boost_0.25,instances_100_1000,1.000000,0.050921
673,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.118487
674,single_elastic_bm25_boost_0.25,instances_100_1000,0.500000,0.149827
675,single_elastic_bm25_boost_0.25,instances_100_1000,0.055556,0.120693
676,single_elastic_bm25_boost_0.25,instances_100_1000,0.000000,0.097450
...,...,...,...,...
298579,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.091356
298580,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.087534
298581,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.000000,0.090200
298582,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,0.333333,0.096017


In [27]:
df_only_mech_queries_overall_per_category = df_only_mech_queries_per_category_thin.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category", "rr"], ascending=False)
print(df_only_mech_queries_overall_per_category.to_string())

                                                                             method_id               class_category        rr      time
1575                       tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_0.1_boost_0.5           instances_100_1000  0.898201  0.117994
63                                                      single_qdrant_sparse_boost_0.5           instances_100_1000  0.895160  0.095525
975                        tuple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.9_boost_0.5           instances_100_1000  0.895001  0.194590
1119                       tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6_boost_0.5           instances_100_1000  0.893371  0.192812
255    triple_fusion_qdrant_dense_0.1_x_qdrant_sparse_0.8_x_elastic_bm25_0.1_boost_0.5           instances_100_1000  0.892541  0.195667
1071                       tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7_boost_0.5           instances_100_1000  0.890872  0.194023
1023                       tuple_fusion_qdrant_d

# Overall - only user queries

In [28]:
df_only_user_queries = df[df["query_id"].str.startswith("user")]
df_only_user_queries

,method_id,user_id,class_id,query_id,class_category,rr,time
0,single_elastic_bm25_boost_0.25,1,20825628,user_query_0,instances_100_1000,0.00,0.110425
1,single_elastic_bm25_boost_0.25,1,20825628,user_query_1,instances_100_1000,0.00,0.071166
2,single_elastic_bm25_boost_0.25,1,20825628,user_query_2,instances_100_1000,0.00,0.051668
3,single_elastic_bm25_boost_0.25,1,20825628,user_query_3,instances_100_1000,0.00,0.048177
4,single_elastic_bm25_boost_0.25,1,20825628,user_query_4,instances_100_1000,0.00,0.049556
...,...,...,...,...,...,...,...
297743,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,23442,user_query_1,instances_100000_1000000000,1.00,0.078431
297744,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,23442,user_query_2,instances_100000_1000000000,1.00,0.069612
297745,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,23442,user_query_3,instances_100000_1000000000,0.00,0.081128
297746,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,23442,user_query_4,instances_100000_1000000000,0.00,0.088692


In [29]:
print(df_only_user_queries[["method_id", "rr", "time"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr      time
158                      tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.75  0.444588  0.171155
164                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.75  0.444284  0.164017
170                      tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1_boost_0.75  0.443438  0.166795
5                                                      single_qdrant_dense_boost_0.75  0.437833  0.167235
152                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.75  0.434964  0.168799
113  triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1_boost_0.75  0.425746  0.168450
146                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.75  0.423870  0.165279
116  triple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.1_x_elastic_bm25_0.1_boost_0.75  0.422524  0.168946
107  triple_fusion_qdrant_dense_0.6_x_qdrant_s

### Overall per category

In [30]:
df_only_user_queries_per_category_thin = df_only_user_queries[["method_id", "class_category", "rr", "time"]]
df_only_user_queries_per_category_thin

,method_id,class_category,rr,time
0,single_elastic_bm25_boost_0.25,instances_100_1000,0.00,0.110425
1,single_elastic_bm25_boost_0.25,instances_100_1000,0.00,0.071166
2,single_elastic_bm25_boost_0.25,instances_100_1000,0.00,0.051668
3,single_elastic_bm25_boost_0.25,instances_100_1000,0.00,0.048177
4,single_elastic_bm25_boost_0.25,instances_100_1000,0.00,0.049556
...,...,...,...,...
297743,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.00,0.078431
297744,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,1.00,0.069612
297745,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,0.00,0.081128
297746,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,instances_100000_1000000000,0.00,0.088692


In [31]:
df_only_user_queries_overall_per_category = df_only_user_queries_per_category_thin.groupby(["method_id", "class_category"], as_index=False).mean().sort_values(by=["class_category", "rr"], ascending=False)
print(df_only_user_queries_overall_per_category.to_string())

                                                                             method_id               class_category        rr      time
1367                      tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1_boost_0.75           instances_100_1000  0.422273  0.167460
1271                      tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.75           instances_100_1000  0.419903  0.170317
1319                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.75           instances_100_1000  0.419316  0.163747
47                                                      single_qdrant_dense_boost_0.75           instances_100_1000  0.416155  0.166111
1223                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.75           instances_100_1000  0.407858  0.168067
1175                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.75           instances_100_1000  0.396451  0.164661
1127                      tuple_fusion_qdrant_de

### Choosing the best match per class of a user

- The idea is that if user writes the 6 queries, and at least one query find the result, the user can use it to model the data structure.
- We want to choose the best query for each of the user assigned classes.
- We can do it by grouping by: method, user, and class.
  - Then we select a maximum for the groups from reciprocal rank.
  - Then the results will be used to again group by a method. 

In [32]:
df_only_user_queries_max_per_user_class = df_only_user_queries[["method_id", "user_id", "class_id", "rr"]].groupby(["method_id", "user_id", "class_id"], as_index=False).max()
df_only_user_queries_max_per_user_class

,method_id,user_id,class_id,rr
0,single_elastic_bm25_boost_0.25,1,34038,0.000000
1,single_elastic_bm25_boost_0.25,1,602884,0.000000
2,single_elastic_bm25_boost_0.25,1,2066754,0.000000
3,single_elastic_bm25_boost_0.25,1,4897819,0.000000
4,single_elastic_bm25_boost_0.25,1,7725634,0.000000
...,...,...,...,...
22171,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,8016240,0.333333
22172,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,11483816,0.500000
22173,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,18593264,0.500000
22174,tuple_fusion_qdrant_sparse_0.9_x_elastic_bm25_...,14,28942331,1.000000


### Average rr with only best ranks per classes of users


In [33]:
print(df_only_user_queries_max_per_user_class[["method_id", "rr"]].groupby(["method_id"], as_index=False).mean().sort_values(by="rr", ascending=False).to_string())

                                                                            method_id        rr
158                      tuple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.3_boost_0.75  0.792347
164                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.75  0.786905
152                      tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.75  0.782377
170                      tuple_fusion_qdrant_dense_0.9_x_qdrant_sparse_0.1_boost_0.75  0.779773
5                                                      single_qdrant_dense_boost_0.75  0.777466
107  triple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.3_x_elastic_bm25_0.1_boost_0.75  0.772406
113  triple_fusion_qdrant_dense_0.7_x_qdrant_sparse_0.2_x_elastic_bm25_0.1_boost_0.75  0.770685
146                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.75  0.769728
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  0.765795
145                       tuple_fusion_q

### Number of classes found per method with only best ranks

- In total we have 56 classes and each class is given to two users.
- Each method have a change of scoring 112 found classes.

In [34]:
df_only_user_queries_max_per_user_class_thin = df_only_user_queries_max_per_user_class[["method_id", "rr"]].copy()
# Round up rr meaning the method found the class.
df_only_user_queries_max_per_user_class_thin["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
# Group the results by method -> for a method, each class is presented twice (since two users provided the queries).
print(df_only_user_queries_max_per_user_class_thin.groupby(["method_id"], as_index=False).sum().sort_values(by="rr", ascending=False).to_string())


                                                                            method_id   rr
98   triple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.4_x_elastic_bm25_0.1_boost_0.75  104
164                      tuple_fusion_qdrant_dense_0.8_x_qdrant_sparse_0.2_boost_0.75  104
70    triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.6_x_elastic_bm25_0.1_boost_0.5  104
46    triple_fusion_qdrant_dense_0.2_x_qdrant_sparse_0.5_x_elastic_bm25_0.3_boost_0.5  104
68   triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.5_x_elastic_bm25_0.2_boost_0.75  104
146                      tuple_fusion_qdrant_dense_0.5_x_qdrant_sparse_0.5_boost_0.75  104
65   triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.4_x_elastic_bm25_0.3_boost_0.75  104
64    triple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.4_x_elastic_bm25_0.3_boost_0.5  104
133                       tuple_fusion_qdrant_dense_0.3_x_qdrant_sparse_0.7_boost_0.5  104
151                       tuple_fusion_qdrant_dense_0.6_x_qdrant_sparse_0.4_boost_0.5  104

### For each method and a class we get the number of successful search queries

- There is 12 tries for each method, since we got 2 users and 6 queries per a user.


In [35]:
df1_method_class_successes_thin = df_only_user_queries[["method_id", "class_id", "rr"]]
df1_method_class_successes_thin = df1_method_class_successes_thin.copy()
df1_method_class_successes_thin["rr"] = df1_method_class_successes_thin["rr"].apply(lambda x: math.ceil(x))
print(df1_method_class_successes_thin.groupby(["class_id", "method_id"], as_index=False).sum().to_string())

        class_id                                                                         method_id  rr
0            532                                                    single_elastic_bm25_boost_0.25   0
1            532                                                     single_elastic_bm25_boost_0.5   0
2            532                                                    single_elastic_bm25_boost_0.75   0
3            532                                                    single_qdrant_dense_boost_0.25   8
4            532                                                     single_qdrant_dense_boost_0.5   8
5            532                                                    single_qdrant_dense_boost_0.75   8
6            532                                                   single_qdrant_sparse_boost_0.25   8
7            532                                                    single_qdrant_sparse_boost_0.5   8
8            532                                                   single

- If we sum this per a class
- We have 66 methods (3 base, 27 tuple, 36 triple) * 3 (for each boost weight), the class appears twice and 6 queries per class.

In [36]:
print(df1_method_class_successes_thin[["class_id", "rr"]].groupby(["class_id"], as_index=False).sum().sort_values(by="rr", ascending=False).to_string())

     class_id    rr
36   11483816  2271
46   23002039  2156
12     130019  2131
22    1348305  2067
38   13442814  2037
28    2819161  2026
29    3007842  2007
39   15052846  1981
48   29654788  1960
25    1478437  1952
16     294414  1862
51   51591359  1763
1       11303  1746
42   19389637  1725
24    1371150  1667
4       30022  1658
20    1134686  1630
27    2319498  1565
50   35922124  1559
41   18593264  1549
45   21191270  1544
19    1076052  1535
33    5783996  1506
17     310899  1485
3       24856  1465
9       47521  1439
0         532  1438
47   28942331  1436
6       34763  1436
49   30325611  1386
31    3621673  1379
37   12890393  1351
23    1364302  1346
5       34038  1258
55  124030970  1245
30    3231690  1195
8       44539  1191
10      79007  1146
2       23442  1120
43   20755546  1116
52   63125083  1110
15     179049  1069
18     602884  1033
53  123523193   921
7       43229   783
14     134556   745
44   20825628   700
21    1195098   628
26    2066754   623


### Classes the best methods did not find from no rerank results

- triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1
- tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6

In [37]:
# Triple fusion triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1

df1_triple_best = df_only_user_queries_max_per_user_class[df_only_user_queries_max_per_user_class["method_id"].str.contains("triple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.5_x_elastic_bm25_0.1")][["method_id", "class_id", "rr"]].copy()
df1_triple_best["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
df1_triple_best[df1_triple_best["rr"] == 0]

,method_id,class_id,rr
9411,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,4897819,0
9412,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,7725634,0
9413,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,18557321,0
9416,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,133036,0
9423,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,123691918,0
9427,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,4897819,0
9437,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,20755546,0
9439,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,123691918,0
9475,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,124714,0
9477,triple_fusion_qdrant_dense_0.4_x_qdrant_sparse...,1195098,0


- The class it did not find at all:
    - 4897819 - role
    - 123691918 - tool

In [38]:
# Tuple fusion tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6

df1_triple_best = df_only_user_queries_max_per_user_class[df_only_user_queries_max_per_user_class["method_id"].str.contains("tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_0.6")][["method_id", "class_id", "rr"]].copy()
df1_triple_best["rr"] = df_only_user_queries_max_per_user_class_thin["rr"].apply(lambda x: math.ceil(x))
df1_triple_best[df1_triple_best["rr"] == 0]

,method_id,class_id,rr
15459,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,4897819,0
15460,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,7725634,0
15461,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,18557321,0
15471,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,123691918,0
15475,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,4897819,0
15485,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,20755546,0
15487,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,123691918,0
15525,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,1195098,0
15563,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,8016240,0
15571,tuple_fusion_qdrant_dense_0.4_x_qdrant_sparse_...,4897819,0


- The classes it did not find at all
    - 4897819 - role
    - 123691918 - tool

Interestingly, they are the same as before. Looking into the queries, we tried them manually, and it did not find them.
Thinking, maybe it was since the things were too abstract, the users have a big problem with generating the queries to.

# The least found classes

In [39]:
df[["class_id", "class_category", "rr"]].groupby(["class_id", "class_category"], as_index=False).mean().sort_values(by="rr", ascending=False)

,class_id,class_category,rr
12,130019,instances_10000_100000,0.928882
51,51591359,instances_10000_100000,0.811794
48,29654788,instances_100000_1000000000,0.802609
29,3007842,instances_1000_10000,0.761753
46,23002039,instances_1000_10000,0.761584
39,15052846,instances_100_1000,0.751415
28,2819161,instances_100_1000,0.736428
45,21191270,instances_100000_1000000000,0.732026
22,1348305,instances_10000_100000,0.718077
24,1371150,instances_100_1000,0.714530
